In [1]:
import numpy as np
import holoviews as hv
from holoviews import opts

In [ ]:
hv.extension('bokeh')

defaults = dict(width=700, height=700, padding=0.1)
hv.opts.defaults(
    opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

### version 3

In [48]:
class Job:
    """
    Container for Job and its attributes.
    Also sets x and y coordinates for each job.
    """
    jobs = {}

    def __init__(self, name, dependencies=None):
        self.name = name
        self.dependencies = dependencies
        
        self.level = 0
        self.x = None

        Job.jobs[self.name] = self

    @classmethod
    def assign_levels(cls):
        """ Set y coordinate for all Job objects. """

        def update_levels(job):
            # setting level of dependencies to be 1 greater then current job:
            for d_on_job in job.dependencies:
                if Job.jobs[d_on_job].level > job.level:
                    pass
                else:
                    Job.jobs[d_on_job].level = job.level + 1
                    # update recursively
                    update_levels(Job.jobs[d_on_job])

        # for each job update levels of its dependencies jobs
        for job_name, job in Job.jobs.items():
            update_levels(job)

    @classmethod
    def set_x(cls):
        """ set x coordinate for all Job objects. """
        x_spacing_between_nodes = 5
        # {level: no_of_jobs}
        xs = {}
        for job in Job.jobs.values():
            if job.level in xs.keys():
                xs[job.level] += 1
            else:
                # if there are no jobs at level "job.level" then now we got one job at that level
                xs[job.level] = 1
            # say this is the 3rd node at this level. Then its x-coordinate = 3*(spacing between nodes)
            job.x = xs[job.level] * x_spacing_between_nodes

    @classmethod
    def print_job_levels(cls):
        for job in Job.jobs.values():
            print('job: {name}, level: {level}'.format(name=job.name, level=job.level))
    
    @classmethod
    def get_edges(cls):
        """
        returns a list of tuples: [(start_node, end_node), ...]
        tuple defines the end-nodes(or end-points) of an edge.
        """
        edges = []
        for job in Job.jobs.values():
            for _ in job.dependencies:
                edges.append((_, job.name))
        return edges


def straight_path(start, end, x, y):
    x_coordinates = np.linspace(x[start], x[end], 100)
    y_coordinates = np.linspace(y[start], y[end], 100)
    return x_coordinates, y_coordinates


def curved_path(start, end, x, y):
    """
    a = ((y[start] - mid_pt_y)**2) / (4*(-2))
    x = (((y - mid_pt_y)**2) / (4*a) ) + (x[start]+2)
    """
    
    curvature = -(y[end] - y[start])
    mid_pt_y = y[start] + abs(y[start] - y[end])/2
    a = ((y[start] - mid_pt_y)**2) / (4*(-curvature))
    
    # 100 y values between start_y and end_y
    y_coordinates = list(np.linspace(y[start], y[end], 100))
    
    # 100 x values corresponding to above y values
    x_coordinates = []
    for y_cord in y_coordinates:
        x_coordinates.append((((y_cord - mid_pt_y)**2) / (4*a) ) + (x[start]+curvature))
    return x_coordinates, y_coordinates


def main(jobs):
    # hv uses node_indexes
    mapping = {index: job_name for index, job_name in enumerate(jobs.keys())}
    reverse_mapping = {job_name: index for index, job_name in enumerate(jobs.keys())}
    
    # because fucking hv uses node_indexes
    hv_jobs = {}
    for k, v in jobs.items():
        hv_jobs[reverse_mapping[k]] = [reverse_mapping[d_job] for d_job in v]
    #hv_jobs = {reverse_mapping[k]: [reverse_mapping[d_job] for d_job in v] for k, v in jobs.items()}
    
    # making Job objects from dictionary
    for jb, dependencies in hv_jobs.items():
        Job(jb, dependencies)
    Job.assign_levels()
    Job.print_job_levels()
    Job.set_x()
    
    # getting edges [(start node, end node), ...]
    edges = Job.get_edges()
    edges_start = [s for s, _ in edges]
    edges_end = [e for _, e in edges]
    
    # getting job name, job's x and job's y for passing to hv
    x = []
    y = []
    node_indices = []
    for job in Job.jobs.values():
        node_indices.append(job.name)
        x.append(job.x)
        y.append(job.level)
    
    
    paths = []
    for c_node_index in node_indices:
        # position of a node
        cn_x, cn_y = x[c_node_index], y[c_node_index]
        
        for end_node in hv_jobs[c_node_index]:
            if x[end_node] == cn_x and abs(y[end_node] - cn_y) > 1:
                x_coordinates, y_coordinates = curved_path(start=c_node_index, end=end_node, x=x, y=y)
            else:
                x_coordinates, y_coordinates = straight_path(start=c_node_index, end=end_node, x=x, y=y)
            paths.append(np.column_stack([x_coordinates, y_coordinates]))
    
    # for hover over names
    nodes = hv.Nodes((x, y, node_indices, [mapping[i] for i in node_indices]), vdims='Job_name')
    
    G = hv.Graph(((edges_start, edges_end), nodes, paths))
    hv.save(G, 'graph.html')


if __name__ == '__main__':
    jobs = {'a': list('bchg'),
            'b': list('ih'),
            'c': list('edfhg'),
            'i': 'h',
            'e': [],
            'd': [],
            'f': list('gtxz'),
            'h': [],
            'g': []
            }
    
    # for jobs with no details
    extend_jobs = {}
    for value in jobs.values():
        if isinstance(value, str):
            value = [value]
        for job in value:
            if job not in jobs.keys():
                extend_jobs[job] = []
    jobs.update(extend_jobs)
    
    main(jobs)


job: 0, level: 0
job: 1, level: 1
job: 2, level: 2
job: 3, level: 0
job: 4, level: 1
job: 5, level: 1
job: 6, level: 1
job: 7, level: 1
job: 8, level: 1
job: 9, level: 1
job: 10, level: 1
job: 11, level: 3
